In [1]:
import os
import sys
sys.path.append('../modules')

import numpy as np
import cv2

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = InteractiveSession(config=config)

from srgan_d import SRGAN
from generator import ImageDataGenerator

Using TensorFlow backend.


In [2]:
r = 2
input_dir = f'../data/train/{r}/low/'
tgt_dir = f'../data/train/{r}/high/'

input_dir_val = f'../data/val/{r}/low/'
tgt_dir_val = f'../data/val/{r}/high/'


test_file = 'UBMk30rjy0o_17675_42.jpg'
test_low_path =  input_dir_val + test_file
test_high_path = tgt_dir_val + test_file
test_out_dir = '../data/out/'

model_dir = '../models/'
vgg_name = 'vgg16_notop.hdf5'
vgg_path = model_dir + vgg_name

input_shape = (67, 120, 3)
tgt_shape = (134, 240, 3)

batch_size = 16

In [3]:
srgan = SRGAN(vgg_path, batch_size, input_shape, tgt_shape)
gen = ImageDataGenerator(input_dir, tgt_dir, batch_size)
gen_valid = ImageDataGenerator(input_dir_val, tgt_dir_val, batch_size)

../modules/srgan_d.py:110: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="generator", inputs=Tensor("in..., outputs=Tensor("ou...)`
  self.shared_generator = Network(input=layers[0], output=layers[-1], name='generator')
../modules/srgan_d.py:139: UserWarning: Update your `Network` call to the Keras 2 API: `Network(name="discriminator", inputs=Tensor("in..., outputs=Tensor("de...)`
  self.shared_discriminator = Network(input=layers[0], output=layers[-1], name='discriminator')
/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
for e in range(101):
    X_low, X_high = gen.flow_from_directory().__next__()
    srgan.train(X_low, X_high)
    if e % 100 == 0:
        print('epoch', e)
        srgan.valid(X_low, X_high)
        
        test_img_low = (cv2.imread(test_low_path) / 255.0)[np.newaxis, :, :, :]
        test_img_high = (cv2.imread(test_high_path) / 255.0)[np.newaxis, :, :, :]

        test_img_sr = srgan.generator.predict(test_img_low)
        test_img_sr_int = (test_img_sr * 255).astype('u1')[0]
        test_out_path = test_out_dir + test_file.split('.')[0] + f'_test_{e}.jpg'
        cv2.imwrite(test_out_path, test_img_sr_int)

epoch 0
d_loss_real:0.8890551924705505, d_loss_fake:3.038712978363037, vgg_loss:4.930268287658691


KeyboardInterrupt: 

In [6]:
out_model_name = '20200516_generator.h5'
srgan.generator.save(model_dir + out_model_name, include_optimizer=False)